# Extracting QJE affiliation data from base tesseract data

First the JSON output file from tezseract is read in and made into a useable dataframe

In [7]:
#Importing the libraries
import json
import pandas as pd
from rapidfuzz import process
import requests

# Load input files from github
tesseract_input = requests.get('https://raw.githubusercontent.com/Matt970204/Affiliations_V2/Clean-online-hosted/1.%20Input%20Data/Tesseract%20affiliation%20output/QJE/QJE_aff_ref_output_1977_2010.json')
json_data = tesseract_input.json()

# Import JSON file
#f = open('/Users/MattJohnson/Desktop/Affiliations/1_Input files and processing input files/1_1_Input files/Tesseract affiliation output/QJE/QJE_aff_ref_output_1977_2010.json')
#json_data = json.load(f)

# Define Journal and Dataframe
Journal_Name = 'QJE'
Reference_Data_List = []

# Extract Json Data into dataframe
for key1, value1 in json_data.items():
    Article_ID = key1
    content_type = value1["content_type"]
    author_list = value1["authors"]
    authorsNo = len(author_list)
    stable_url = value1["stable_url"]
    ref_string_found = ''
    ref_string_raw = ''
    list = []
    for key2 in value1["affiliations"]:
        #print(key2)
        if key2 == "found":
            Ref_type = key2
            ref_string_found = value1["affiliations"][key2]
            #print("found")
        else: 
            for key3 in value1["affiliations"]["raw"]:
                    list.append(key3)
                    Ref_type = key2
                    #print(key1, "   ",len(value1["affiliations"]["raw"]))
                    #print("raw", key3)
                    #ref_string_raw = value1["affiliations"][key2][key3]
                    #print(ref_string)
            #print(list)
            #print(list[-1])
            if len(list) >0:
                raw = value1["affiliations"][key2][list[-1]]
                #print(raw)
                #print(type(raw))
                reverse = raw[::-1]
                #print(reverse)
                ref_string_raw = "".join(reverse)

        
    for key2 in value1["affiliations"]:
        #print(key2)
        if key2 == "found":
            Ref_type = key2
            break
        else: 
            Ref_type = key2

    Reference_Data_List.append({'Journal_Name' : Journal_Name,
    'Authors_count' : authorsNo, 
    'Authors_list' : author_list,
    'Article_ID' : Article_ID, 'content_type' : content_type,
    'stable_url' : stable_url, 
    'Ref_type': Ref_type,  
    'ref_string_found':ref_string_found,
     'ref_string_raw': ref_string_raw})

# Create a dataframe from the extracted data
QJE_aff_ref_output_1940_2010 = pd.DataFrame.from_records(Reference_Data_List)  

#Basic view of data
print(QJE_aff_ref_output_1940_2010.head())
print(QJE_aff_ref_output_1940_2010.shape)

#Use line below to send data to excel to view full file
#QJE_aff_ref_output_1940_2010.to_excel('/Users/MattJohnson/Desktop/Affiliations/2_Extracting affilitations/3_Consolidation/QJE/QJE_aff_ref_output_1940_2010_Initial.xlsx')        

  Journal_Name  Authors_count  \
0          QJE              2   
1          QJE              2   
2          QJE              2   
3          QJE              3   
4          QJE              2   

                                        Authors_list Article_ID content_type  \
0               [Nicola Gennaioli,  Andrei Shleifer]   40961010      Article   
1                 [Dean P. Foster,  H. Peyton Young]   40961011      Article   
2                  [Eric D. Gould,  Esteban F. Klor]   40961012      Article   
3  [Daron Acemoglu,  Georgy Egorov,  Konstantin S...   40961013      Article   
4                  [Shaohua Chen,  Martin Ravallion]   40961014      Article   

                              stable_url Ref_type  \
0  https://www.jstor.org/stable/40961010    found   
1  https://www.jstor.org/stable/40961011    found   
2  https://www.jstor.org/stable/40961012    found   
3  https://www.jstor.org/stable/40961013    found   
4  https://www.jstor.org/stable/40961014    found   

 

Next the Unique list of universities is read in, this is used as a database to check the affiliton string against using rapidfuzz. 

In [3]:
#testdata = RES_1940_2010.loc[0:20].reset_index(drop = True)
testdata = QJE_aff_ref_output_1940_2010

for i in range(len(testdata)):
        testdata.loc[i,['ref_string_found_clean']] = testdata.iloc[i]['ref_string_found'].strip().title()
        testdata.loc[i,['ref_string_raw_clean']] = testdata.iloc[i]['ref_string_raw'].strip().title()

print(testdata.columns.values)

['Journal_Name' 'Authors_count' 'Authors_list' 'Article_ID' 'content_type'
 'stable_url' 'Ref_type' 'ref_string_found' 'ref_string_raw'
 'ref_string_found_clean' 'ref_string_raw_clean']


In [4]:
#Reading in the master file and correctly formatting the article ID to match back onto the meltef file

#Load in master from Github
QJE_master = pd.read_excel("https://raw.githubusercontent.com/Matt970204/Affiliations_V2/Clean-online-hosted/1.%20Input%20Data/Processed/QJE_processed.xlsx")

#Local store of AER_master
#QJE_master = pd.read_excel('/Users/MattJohnson/Desktop/Affiliations/1_Input files and processing input files/1_1_Input files/Master lists/Processed/QJE_processed.xlsx')

for i in range(len(QJE_master)):
    QJE_master.loc[i,['Article_ID']] = QJE_master.loc[i,['stable_url']].to_string().rsplit('/',1)[-1]

QJE_master_condenced  = QJE_master[['Article_ID','year']]
QJE_master_condenced['year'] = QJE_master_condenced['year'].astype('int')
QJE_master_condenced.head()

/var/folders/23/ft9qf9c11rj_vqhbjml910f40000gn/T/ipykernel_70910/3127414697.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  QJE_master_condenced['year'] = QJE_master_condenced['year'].astype('int')


,Article_ID,year
0,26372672,2016
1,26372673,2016
2,26372674,2016
3,26372675,2016
4,26372676,2016


In [5]:
# Merging the masterfile onto the new melted file to get additional data
testdata_merged = testdata.merge(QJE_master_condenced, how = 'left', on = 'Article_ID')
testdata_merged = testdata_merged.sort_values(by=['year'])
testdata = testdata_merged.reset_index(drop = True)

In [6]:
# Excel output to get an overview of the data
#testdata.head
#testdata.to_excel('/Users/MattJohnson/Desktop/Affiliations/2_Extracting affilitations/3_consolidation/QJE/QJE_aff_ref_output_1940_2010_affs_out setup.xlsx') 
print(testdata.head())

  Journal_Name  Authors_count                         Authors_list Article_ID  \
0          QJE              1                        [Bent Hansen]    1885985   
1          QJE              1                   [Charles R. Laine]    1885981   
2          QJE              1                   [Richard Lecomber]    1885980   
3          QJE              2  [John C. Panzar,  Robert D. Willig]    1885979   
4          QJE              1                    [Elhanan Helpman]    1885978   

  content_type                            stable_url Ref_type  \
0      Article  https://www.jstor.org/stable/1885985      raw   
1      Article  https://www.jstor.org/stable/1885981    found   
2      Article  https://www.jstor.org/stable/1885980    found   
3      Article  https://www.jstor.org/stable/1885979    found   
4      Article  https://www.jstor.org/stable/1885978      raw   

                                    ref_string_found  \
0                                                      
1  TABLE I

In [8]:
testdata2 = testdata

In [10]:
# Creating a dictionary for consolidation
# Read in from Excel sheet, 2 columns first one with incorrect spelling and second one with correct spellig

cities = pd.read_excel("https://raw.githubusercontent.com/Matt970204/Affiliations_V2/Clean-online-hosted/1.%20Input%20Data/Consolidated%20Affiliations/Consolidated%20Affiliations.xlsx", 'Cities')
#cities = pd.read_excel('/Users/MattJohnson/Library/CloudStorage/OneDrive-UniversityofCapeTown/Masters_thesis/Data checks/consolidated list Final Use.xlsx', 'Cities')
cities = cities[['Incorrect spelling', 'Correct spelling']]
#print(cities.shape)

comma_in_name = pd.read_excel("https://raw.githubusercontent.com/Matt970204/Affiliations_V2/Clean-online-hosted/1.%20Input%20Data/Consolidated%20Affiliations/Consolidated%20Affiliations.xlsx", 'Comma in name')
#comma_in_name = pd.read_excel('/Users/MattJohnson/Library/CloudStorage/OneDrive-UniversityofCapeTown/Masters_thesis/Data checks/consolidated list Final Use.xlsx', 'Comma in name')
comma_in_name = comma_in_name[['Incorrect spelling', 'Correct spelling']]
#print(comma_in_name.shape)

misspellings = pd.read_excel("https://raw.githubusercontent.com/Matt970204/Affiliations_V2/Clean-online-hosted/1.%20Input%20Data/Consolidated%20Affiliations/Consolidated%20Affiliations.xlsx", 'Misspellings')
#misspellings = pd.read_excel('/Users/MattJohnson/Library/CloudStorage/OneDrive-UniversityofCapeTown/Masters_thesis/Data checks/consolidated list Final Use.xlsx', 'Misspellings')
misspellings = misspellings[['Incorrect spelling', 'Correct spelling']]

colleges = pd.read_excel("https://raw.githubusercontent.com/Matt970204/Affiliations_V2/Clean-online-hosted/1.%20Input%20Data/Consolidated%20Affiliations/Consolidated%20Affiliations.xlsx", 'Colleges')
#colleges = pd.read_excel('/Users/MattJohnson/Library/CloudStorage/OneDrive-UniversityofCapeTown/Masters_thesis/Data checks/consolidated list Final Use.xlsx', 'Colleges')
colleges = colleges[['Incorrect spelling', 'Correct spelling']]

colleges_full = pd.read_excel("https://raw.githubusercontent.com/Matt970204/Affiliations_V2/Clean-online-hosted/1.%20Input%20Data/Consolidated%20Affiliations/Consolidated%20Affiliations.xlsx", 'Colleges Full')
#colleges_full = pd.read_excel('/Users/MattJohnson/Library/CloudStorage/OneDrive-UniversityofCapeTown/Masters_thesis/Data checks/consolidated list Final Use.xlsx', 'Colleges Full')
colleges_full = colleges_full[['Incorrect spelling', 'Correct spelling']]

correct = pd.read_excel("https://raw.githubusercontent.com/Matt970204/Affiliations_V2/Clean-online-hosted/1.%20Input%20Data/Consolidated%20Affiliations/Consolidated%20Affiliations.xlsx", 'Correct')
#correct = pd.read_excel('/Users/MattJohnson/Desktop/Affiliations/2_Extracting affilitations/2_Fuzzy matching/University list/consolidated list.xlsx', 'Correct')
correct = correct[['Incorrect spelling', 'Correct spelling']]

frames = [cities, misspellings, comma_in_name,colleges_full ,colleges,correct]
consolidated_affiliations = pd.concat(frames)
print(consolidated_affiliations.shape)

for i in range(len(consolidated_affiliations)):
    #Replaceing title case and stripping leading space
    #affs.iat[i,0] = affs.iat[i,0].strip().title()
    consolidated_affiliations.iloc[i]['Incorrect spelling'] = consolidated_affiliations.iloc[i]['Incorrect spelling'].strip().title()
    #affs.iat[i,0] = affs.iat[i,0].strip().upper()

consolidated_affiliations_dict = consolidated_affiliations.set_index('Incorrect spelling').to_dict()['Correct spelling']
print(consolidated_affiliations_dict)

(1714, 2)
{'Ames, Iowa': 'Ames Iowa United States (City)', 'Ames, Iowa, Usa': 'Ames Iowa United States (City)', 'Antwerp': 'Antwerp Belguim (City)', 'Auckland, New Zealand': 'Auckland New Zealand (City)', 'Auckland, Nz': 'Auckland New Zealand (City)', 'Baghdad': 'Baghdad Iraq (City)', 'Baltimore': 'Baltimore Maryland (City)', 'Baltimore, Usa Kzlvin J Lancaster': 'Baltimore Maryland (City)', 'Belfast': 'Belfast Ireland (City)', 'Berkeley': 'Berkeley California United States (City)', 'Berkeley, California': 'Berkeley California United States (City)', 'Birmingham': 'Birmingham United Kingdom (City)', 'Birmingham And Ames, Iowa': 'Birmingham United Kingdom (City)', 'Bloomington, Indiana': 'Bloomington Indiana United States (City)', 'Bristol': 'Bristol United Kingdom (City)', 'Brooklyn, New York': 'Brooklyn New York United States (City)', 'Brunswick, Maine Davidp Braybrooke': 'Brunswick Maine United States (City)', 'Brussels': 'Brussels Belgium (City)', 'Brussels, Belgium': 'Brussels Belgiu

In [12]:
# Load in the list of unique Universites from github stored in 2_fuzzy matching

Affiliation_choices = consolidated_affiliations['Incorrect spelling'].values.tolist()
print(Affiliation_choices)


['Ames, Iowa', 'Ames, Iowa, Usa', 'Antwerp', 'Auckland, New Zealand', 'Auckland, Nz', 'Baghdad', 'Baltimore', 'Baltimore, Usa Kzlvin J Lancaster', 'Belfast', 'Berkeley', 'Berkeley, California', 'Birmingham', 'Birmingham And Ames, Iowa', 'Bloomington, Indiana', 'Bristol', 'Brooklyn, New York', 'Brunswick, Maine Davidp Braybrooke', 'Brussels', 'Brussels, Belgium', 'California', 'Cambridge', 'Cambridge ,', 'Cambridge And Salisbury, Rhodesia', 'Cambridge Bungess Cameron', 'Cambridge Nicholas Kaaldom', 'Cambridge, Mass', 'Cambridge, Mass  And Washington, Dc', 'Cambridge, Mass Rosert M Solow', 'Cambridge, Mass Rosert M Solow,', 'Cambridge, Massachusetts', 'Cambridge, Massachusetts', 'Cambridge, Massachusetts *', 'Canberra', 'Cardiff', 'Chicago', 'Chicago Lzster G Telser', 'Chicago, Illinoi', 'Chicago, Illinois', 'Chicago, Illinois', 'Chicago, Illinois, Usa', 'Columbus, Ohio Beénjamin Caplan', 'Diakarta, Indonesia *', 'Durham, N Carolina', 'Durham, North Carolina', 'Durham, North Carolina Lio

In [13]:
import re
from rapidfuzz import fuzz
import math

for x in range(len(testdata2)):
    str1 = testdata2.iloc[x]['ref_string_found_clean']

#Splitting up the string into authors lines
    ref_split = re.split(r"\n",str1)
    #Removing blank lines from the split
    while("" in ref_split):
        ref_split.remove("")

    author_count = 0
    for j in range(len(ref_split)):
        for z in Affiliation_choices:
            ratio  = fuzz.partial_ratio(ref_split[j], z)
            if ratio > 95:
                testdata2.loc[x, str(author_count)+'_author_affs'] = ref_split[j]
                print(testdata2.loc[x, str(author_count)+'_author_affs'])
                author_count = author_count+1
                break
    print(x)

    

0
1
2
Bell Laboratories
3
4
5
6
7
Data Resources , Incorporated And
Harvard University
8
York University
9
10
11
12
Columbia University
13
14
Àarhus Universitet, Denmark, And University Of Essex, United
Kingdom
15
16
Harvard University
California Institute Of Technology
University Of Arizona
17
Churchill College, Cambridge
18
19
20
Massachusetts Institute Of Technology
21
22
23
24
25
26
Board Of Governors Of The Federal, Reserve System
University Of Massachusetts, Boston
27
28
29
30
31
32
33
Board Of Governors Of The Federal Reserve System
34
Federal Reserve Bank Of San Francisco
35
Harvard University
Stanford University
36
37
School For Advanced Studies In The Social Sciences, And Cepremap,
Paris
38
39
40
University Of Virginia
Australian National University
41
42
43
State University Of New York At Albany
44
45
The Hebrew University
46
47
48
49
50
51
52
53
54
55
56
57
58
'Tel-Aviv University, Israel
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
6. The Federal Re

In [14]:
testdata = testdata2[['Journal_Name' ,'Authors_count', 'Authors_list', 'Article_ID', 'content_type',
 'stable_url', 'Ref_type', 'year',
 'ref_string_found' ,
 'ref_string_found_clean',
 'ref_string_raw',
 'ref_string_found_clean' ,
 'ref_string_raw_clean' ,
 '0_author_affs', 
 '1_author_affs',
 '2_author_affs',
 '3_author_affs',
 '4_author_affs' ]]

In [15]:
print(testdata.columns.values)

['Journal_Name' 'Authors_count' 'Authors_list' 'Article_ID' 'content_type'
 'stable_url' 'Ref_type' 'year' 'ref_string_found'
 'ref_string_found_clean' 'ref_string_raw' 'ref_string_found_clean'
 'ref_string_raw_clean' '0_author_affs' '1_author_affs' '2_author_affs'
 '3_author_affs' '4_author_affs']


In [16]:
import numpy as np
import re
testdata = testdata.replace(np.nan, '', regex=True)
testdata["4_author_affs"] = ""
testdata["5_author_affs"] = ""

testdata["0_author_affs_consolidated"] = ""
testdata["0_author_affs_consolidated_0"] = ""
testdata["0_author_affs_consolidated_1"] = ""
testdata["0_author_affs_consolidated_2"] = ""
testdata["0_author_affs_consolidated_3"] = ""
testdata["0_author_affs_consolidated_4"] = ""

testdata["1_author_affs_consolidated"] = ""
testdata["1_author_affs_consolidated_0"] = ""
testdata["1_author_affs_consolidated_1"] = ""
testdata["1_author_affs_consolidated_2"] = ""
testdata["1_author_affs_consolidated_3"] = ""
testdata["1_author_affs_consolidated_4"] = ""

testdata["2_author_affs_consolidated"] = ""
testdata["2_author_affs_consolidated_0"] = ""
testdata["2_author_affs_consolidated_1"] = ""
testdata["2_author_affs_consolidated_2"] = ""
testdata["2_author_affs_consolidated_3"] = ""
testdata["2_author_affs_consolidated_4"] = ""

testdata["3_author_affs_consolidated"] = ""
testdata["3_author_affs_consolidated_0"] = ""
testdata["3_author_affs_consolidated_1"] = ""
testdata["3_author_affs_consolidated_2"] = ""
testdata["3_author_affs_consolidated_3"] = ""
testdata["3_author_affs_consolidated_4"] = ""

testdata["4_author_affs_consolidated"] = ""
testdata["4_author_affs_consolidated_0"] = ""
testdata["4_author_affs_consolidated_1"] = ""
testdata["4_author_affs_consolidated_2"] = ""
testdata["4_author_affs_consolidated_3"] = ""

testdata["5_author_affs_consolidated"] = ""
testdata["5_author_affs_consolidated_0"] = ""

for i in range(len(testdata)):
#for i in range(1533,1534):
#for i in range(3715,3716):
    authors = testdata.iloc[i]['Authors_list']
    for auth_no in range(len(authors)):
        #print(testdata.iloc[i]['0_author_affs'])
        testdata.loc[i,str(auth_no)+'_author_affs'] =  re.sub(r'The |\.|\'|\"|\:','',testdata.iloc[i][str(auth_no)+'_author_affs']).title()
        print(testdata.loc[i,str(auth_no)+'_author_affs'])
        #print(testdata.iloc[i]['0_author_affs'])
        list = []

    #Loop through the list of incorrect spellings and replace with corrected spellings
        for key in consolidated_affiliations_dict.keys():
            #testdata.loc[i,'0_author_affs_consolidated']  = testdata.iloc[i]['0_author_affs'].replace(key, consolidated_affiliations_dict[key])

            if key in testdata.iloc[i][str(auth_no)+'_author_affs']:
                #print(testdata.iloc[i][str(auth_no)+'_author_affs'])
                list.append(key)
                #print(len(list))
                print((key))
                #print(consolidated_affiliations_dict[key])
                print(list)

    #If there is more than 1 match in the consolidation it needs to match on the longer one, solves for college issue
                if len(list) <= 1:
                    testdata.loc[i,str(auth_no)+'_author_affs_consolidated']  = re.sub(key, consolidated_affiliations_dict[key],testdata.iloc[i][str(auth_no)+'_author_affs']).strip().rstrip(',')
                else:
                    print(consolidated_affiliations_dict[list[0]])
                    print(consolidated_affiliations_dict[list[1]])
                    # For colleges
                    if len(list) ==2:
                        if(len(consolidated_affiliations_dict[list[0]]) == len(consolidated_affiliations_dict[list[1]])):
                            #print("HERE")
                            if len(list[0]) > len(list[1]):
                                print("If")
                                testdata.loc[i,str(auth_no)+'_author_affs_consolidated']  = re.sub(list[0], consolidated_affiliations_dict[list[0]],testdata.iloc[i][str(auth_no)+'_author_affs']).strip().rstrip(',')
                                #print(list[0])
                            # Full  University Of Illinois System Urbana-Champaign issue
                            elif len(list[1]) > len(list[0]):
                                print("Elif")
                                testdata.loc[i,str(auth_no)+'_author_affs_consolidated']  = re.sub(list[1], consolidated_affiliations_dict[list[1]],testdata.iloc[i][str(auth_no)+'_author_affs']).strip().rstrip(',')
                            else:
                                print("else")
                                first_fix = re.sub(list[1], consolidated_affiliations_dict[list[1]],testdata.iloc[i][str(auth_no)+'_author_affs']).strip().rstrip(',') 
                                testdata.loc[i,str(auth_no)+'_author_affs_consolidated']  = re.sub(key, consolidated_affiliations_dict[key],first_fix).strip().rstrip(',') 
                        else:
                            first_fix = re.sub(list[0], consolidated_affiliations_dict[list[0]],testdata.iloc[i][str(auth_no)+'_author_affs']).strip().rstrip(',') 
                            #print(first_fix)
                            testdata.loc[i,str(auth_no)+'_author_affs_consolidated']  = re.sub(key, consolidated_affiliations_dict[key],first_fix).strip().rstrip(',') 
# ['University Of California At Berkeley', 'California At Berkeley', 'Department Of Economics, University Of California At Berkeley', 'University Of California']
                    else:
                        print('long len')
                        if(len(consolidated_affiliations_dict[list[0]]) == len(consolidated_affiliations_dict[list[1]])== len(consolidated_affiliations_dict[list[2]])):
                            if(len(list[0]) > len(list[1]) and len(list[0]) > len(list[2])):
                                print("If")
                                testdata.loc[i,str(auth_no)+'_author_affs_consolidated']  = re.sub(list[0], consolidated_affiliations_dict[list[0]],testdata.iloc[i][str(auth_no)+'_author_affs']).strip().rstrip(',')
                                #print(list[0])
                            # Full  University Of Illinois System Urbana-Champaign issue
                            elif(len(list[1]) > len(list[0]) and len(list[1]) > len(list[2])):
                                print("Elif")
                                testdata.loc[i,str(auth_no)+'_author_affs_consolidated']  = re.sub(list[1], consolidated_affiliations_dict[list[1]],testdata.iloc[i][str(auth_no)+'_author_affs']).strip().rstrip(',')
                            elif(len(list[2]) > len(list[0]) and len(list[2]) > len(list[1])):
                                print("Elif")
                                testdata.loc[i,str(auth_no)+'_author_affs_consolidated']  = re.sub(list[2], consolidated_affiliations_dict[list[1]],testdata.iloc[i][str(auth_no)+'_author_affs']).strip().rstrip(',')
                            else:
                                print("else")
                                first_fix = re.sub(list[1], consolidated_affiliations_dict[list[1]],testdata.iloc[i][str(auth_no)+'_author_affs']).strip().rstrip(',') 
                                testdata.loc[i,str(auth_no)+'_author_affs_consolidated']  = re.sub(key, consolidated_affiliations_dict[key],first_fix).strip().rstrip(',') 
                        else:
                            first_fix = re.sub(list[0], consolidated_affiliations_dict[list[0]],testdata.iloc[i][str(auth_no)+'_author_affs']).strip().rstrip(',') 
                            #print(first_fix)
                            testdata.loc[i,str(auth_no)+'_author_affs_consolidated']  = re.sub(key, consolidated_affiliations_dict[key],first_fix).strip().rstrip(',') 
            
            # print(i)

                #print(re.sub(key, consolidated_affiliations_dict[key],testdata.iloc[i]['0_author_affs']))
            #else:
            #    testdata.loc[i,'0_author_affs_consolidated'] = ''
                #print(testdata.iloc[i]['0_author_affs_consolidated'])
        #testdata.loc[i,['1_author_affs']] =  re.sub(r'\.','',testdata.iloc[i]['1_author_affs'])
        #testdata.loc[i,['2_author_affs']] =  re.sub(r'\.','',testdata.iloc[i]['2_author_affs'])
        #if(testdata.iloc[i][str(auth_no)+'_author_affs_consolidated']):
        #    print("exists")
        print(i)
        print('Full ',testdata.loc[i,str(auth_no)+'_author_affs_consolidated'])
        #else: 
        #    print("non existant")
        auths_affs_split_And = re.split(' And ', testdata.iloc[i][str(auth_no)+'_author_affs_consolidated'])
        #print(auths_affs_split_And)
        #print(len(auths_affs_split_And))

        for aff in range(len(auths_affs_split_And)):
            testdata.loc[i,str(auth_no)+'_author_affs_consolidated_'+str(aff)]  = auths_affs_split_And[aff].strip().rstrip(',') 
            print('Auth', str(auth_no), str(aff) , testdata.loc[i,str(auth_no)+'_author_affs_consolidated_'+str(aff)])

        auths_affs_split_comma = re.split(',', testdata.iloc[i][str(auth_no)+'_author_affs_consolidated_0'])
        #print(auths_affs_split_And)
        #print(len(auths_affs_split_And))
        if len(auths_affs_split_comma) == 2:
            for affcomma in range(len(auths_affs_split_comma)):
                testdata.loc[i,str(auth_no)+'_author_affs_consolidated_0']  = auths_affs_split_comma[0].strip().rstrip(',')
                if  testdata.loc[i,str(auth_no)+'_author_affs_consolidated_1'] == '':
                    testdata.loc[i,str(auth_no)+'_author_affs_consolidated_1']  = auths_affs_split_comma[1].strip().rstrip(',')
                else:
                    testdata.loc[i,str(auth_no)+'_author_affs_consolidated_2']  = auths_affs_split_comma[1].strip().rstrip(',')
                print('Auth', str(auth_no), str(affcomma) , testdata.loc[i,str(auth_no)+'_author_affs_consolidated_'+str(affcomma)])
                print(testdata.loc[i,str(auth_no)+'_author_affs_consolidated_2'])
        if len(auths_affs_split_comma) == 3:
            for affcomma in range(len(auths_affs_split_comma)):
                testdata.loc[i,str(auth_no)+'_author_affs_consolidated_0']  = auths_affs_split_comma[0].strip().rstrip(',')
                if  testdata.loc[i,str(auth_no)+'_author_affs_consolidated_1'] == '':
                    testdata.loc[i,str(auth_no)+'_author_affs_consolidated_1']  = auths_affs_split_comma[1].strip().rstrip(',')
                else:
                    testdata.loc[i,str(auth_no)+'_author_affs_consolidated_2']  = auths_affs_split_comma[1].strip().rstrip(',')
                    testdata.loc[i,str(auth_no)+'_author_affs_consolidated_3']  = auths_affs_split_comma[2].strip().rstrip(',')
                print('Auth', str(auth_no), str(affcomma) , testdata.loc[i,str(auth_no)+'_author_affs_consolidated_'+str(affcomma)])
                print(testdata.loc[i,str(auth_no)+'_author_affs_consolidated_2'])
        if len(auths_affs_split_comma) == 4:
            for affcomma in range(len(auths_affs_split_comma)):
                testdata.loc[i,str(auth_no)+'_author_affs_consolidated_0']  = auths_affs_split_comma[0].strip().rstrip(',')
                if  testdata.loc[i,str(auth_no)+'_author_affs_consolidated_1'] == '':
                    testdata.loc[i,str(auth_no)+'_author_affs_consolidated_1']  = auths_affs_split_comma[1].strip().rstrip(',')
                else:
                    testdata.loc[i,str(auth_no)+'_author_affs_consolidated_2']  = auths_affs_split_comma[1].strip().rstrip(',')
                    testdata.loc[i,str(auth_no)+'_author_affs_consolidated_3']  = auths_affs_split_comma[2].strip().rstrip(',')
                    testdata.loc[i,str(auth_no)+'_author_affs_consolidated_4']  = auths_affs_split_comma[3].strip().rstrip(',')

                print('Auth', str(auth_no), str(affcomma) , testdata.loc[i,str(auth_no)+'_author_affs_consolidated_'+str(affcomma)])
                print(testdata.loc[i,str(auth_no)+'_author_affs_consolidated_2'])

    print(i)


0
Full  
Auth 0 0 
0

1
Full  
Auth 0 0 
1

2
Full  
Auth 0 0 
2
Bell Laboratories
Bell Laboratories
['Bell Laboratories']
3
Full  Bell Laboratories
Auth 0 0 Bell Laboratories

3
Full  
Auth 1 0 
3

4
Full  
Auth 0 0 
4

5
Full  
Auth 0 0 

5
Full  
Auth 1 0 
5

6
Full  
Auth 0 0 

6
Full  
Auth 1 0 
6

7
Full  
Auth 0 0 
7
Data Resources , Incorporated And
8
Full  
Auth 0 0 
8
York University
York University
['York University']
9
Full  York University
Auth 0 0 York University
9

10
Full  
Auth 0 0 

10
Full  
Auth 1 0 
10

11
Full  
Auth 0 0 
11

12
Full  
Auth 0 0 

12
Full  
Auth 1 0 

12
Full  
Auth 2 0 
12
Columbia University
Columbia University
['Columbia University']
13
Full  Columbia University
Auth 0 0 Columbia University
13

14
Full  
Auth 0 0 

14
Full  
Auth 1 0 
14
Àarhus Universitet, Denmark, And University Of Essex, United
Àarhus Universitet, Denmark
['Àarhus Universitet, Denmark']
University Of Essex, United
['Àarhus Universitet, Denmark', 'University Of Essex, United'

In [17]:
print(testdata.columns.values)

['Journal_Name' 'Authors_count' 'Authors_list' 'Article_ID' 'content_type'
 'stable_url' 'Ref_type' 'year' 'ref_string_found'
 'ref_string_found_clean' 'ref_string_raw' 'ref_string_found_clean'
 'ref_string_raw_clean' '0_author_affs' '1_author_affs' '2_author_affs'
 '3_author_affs' '4_author_affs' '5_author_affs'
 '0_author_affs_consolidated' '0_author_affs_consolidated_0'
 '0_author_affs_consolidated_1' '0_author_affs_consolidated_2'
 '0_author_affs_consolidated_3' '0_author_affs_consolidated_4'
 '1_author_affs_consolidated' '1_author_affs_consolidated_0'
 '1_author_affs_consolidated_1' '1_author_affs_consolidated_2'
 '1_author_affs_consolidated_3' '1_author_affs_consolidated_4'
 '2_author_affs_consolidated' '2_author_affs_consolidated_0'
 '2_author_affs_consolidated_1' '2_author_affs_consolidated_2'
 '2_author_affs_consolidated_3' '2_author_affs_consolidated_4'
 '3_author_affs_consolidated' '3_author_affs_consolidated_0'
 '3_author_affs_consolidated_1' '3_author_affs_consolidated_2'


In [18]:
testdata1 = testdata[['Journal_Name' ,'Authors_count', 'Authors_list', 'Article_ID', 'content_type',
'stable_url', 'Ref_type' , 'year',
'ref_string_raw' ,
'ref_string_raw_clean',
'ref_string_found',
'ref_string_found_clean' ,
 '0_author_affs', 
 '0_author_affs_consolidated',
 '0_author_affs_consolidated_0',
 '0_author_affs_consolidated_1', 
 '0_author_affs_consolidated_2', 
 '0_author_affs_consolidated_3', 
 '0_author_affs_consolidated_4', 
 '1_author_affs', 
 '1_author_affs_consolidated',
 '1_author_affs_consolidated_0',
 '1_author_affs_consolidated_1',
 '1_author_affs_consolidated_2',
 '1_author_affs_consolidated_3',
 '2_author_affs',
 '2_author_affs_consolidated',
 '2_author_affs_consolidated_0',
 '2_author_affs_consolidated_1', 
 '2_author_affs_consolidated_2', 
 '2_author_affs_consolidated_3', 
 '3_author_affs',
 '3_author_affs_consolidated', 
 '3_author_affs_consolidated_0',
 '3_author_affs_consolidated_1',
 '3_author_affs_consolidated_2',
 '4_author_affs',
 '4_author_affs_consolidated',
 '4_author_affs_consolidated_0',
 '5_author_affs', 
 '5_author_affs_consolidated']]

In [19]:
# Excel output to get an overview of the data
testdata1.head
testdata1.to_excel('/Users/MattJohnson/Desktop/Affiliations/2. Extracting affilitations from Tesseract/2.1. Consolidation/2.1.A. Data created in 2.1/QJE first pass tesseract affiliations.xlsx') 